In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression

In [2]:
DATA_PATH = "skygeni_sales_data.csv"
WIN_RATE_ALERT_THRESHOLD = 0.40

Data Loader

In [3]:
def load_data(path):

    df = pd.read_csv(path)

    df['created_date'] = pd.to_datetime(df['created_date'], errors='coerce')
    df['closed_date'] = pd.to_datetime(df['closed_date'], errors='coerce')

    df['outcome'] = df['outcome'].str.lower().str.strip()

    return df

FEATURE ENGINEERING

In [4]:
def add_features(df):

    df['sales_cycle_days'] = (
        df['closed_date'] - df['created_date']
    ).dt.days

    df['deal_velocity'] = (
        df['deal_amount'] /
        df['sales_cycle_days'].replace(0, 1)
    )

    df['is_large_deal'] = (
        df['deal_amount'] > df['deal_amount'].median()
    ).astype(int)

    return df

EXECUTIVE METRICS

In [5]:
def executive_metrics(df):

    metrics = {}

    metrics['total_pipeline_value'] = df['deal_amount'].sum()
    metrics['avg_deal_size'] = df['deal_amount'].mean()
    metrics['win_rate'] = (df['outcome'] == 'won').mean()
    metrics['avg_sales_cycle'] = df['sales_cycle_days'].mean()

    return metrics

DEAL RISK MODEL

In [6]:
def train_deal_risk_model(df):

    features = ['industry', 'region', 'product_type', 'lead_source']

    X = df[features]
    y = (df['outcome'] == 'lost').astype(int)

    encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    X_encoded = encoder.fit_transform(X)

    model = LogisticRegression(max_iter=1000)
    model.fit(X_encoded, y)

    return model, encoder, features

WIN DRIVER ANALYSIS

In [7]:
def win_driver_analysis(df):

    features = ['industry', 'region', 'product_type', 'lead_source']

    X = df[features]
    y = (df['outcome'] == 'won').astype(int)

    encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    X_encoded = encoder.fit_transform(X)

    model = LogisticRegression(max_iter=1000)
    model.fit(X_encoded, y)

    names = encoder.get_feature_names_out(features)

    importance = pd.DataFrame({
        'feature': names,
        'impact': model.coef_[0]
    }).sort_values(by='impact')

    return importance

REVENUE FORECAST

In [8]:
def revenue_forecast(df):

    recent_win_rate = (df['outcome'] == 'won').mean()

    open_pipeline = df[df['outcome'] != 'won']['deal_amount'].sum()

    forecast_revenue = open_pipeline * recent_win_rate

    return forecast_revenue

 ANOMALY DETECTION

In [9]:
def detect_pipeline_anomalies(df):

    anomalies = []

    avg_deal = df['deal_amount'].mean()

    large_spikes = df[df['deal_amount'] > avg_deal * 3]

    if len(large_spikes) > 0:
        anomalies.append("Unusual high-value deals detected")

    return anomalies

ALERT SYSTEM

In [10]:
def generate_alerts(df, metrics):

    alerts = []

    if metrics['win_rate'] < WIN_RATE_ALERT_THRESHOLD:
        alerts.append("Win rate below healthy threshold")

    slow_deals = df[
        df['sales_cycle_days'] >
        df['sales_cycle_days'].median()
    ]

    if len(slow_deals) > len(df) * 0.3:
        alerts.append("Too many slow-moving deals")

    return alerts

MAIN PIPELINE

In [11]:
def run():

    print("\nLoading Data...")
    df = load_data(DATA_PATH)

    print("Adding Features...")
    df = add_features(df)

    print("\nExecutive Metrics:")
    metrics = executive_metrics(df)
    print(metrics)

    print("\nTraining Deal Risk Model...")
    train_deal_risk_model(df)

    print("\nRunning Win Driver Analysis...")
    drivers = win_driver_analysis(df)
    print(drivers.head())

    print("\nRevenue Forecast...")
    forecast = revenue_forecast(df)
    print(forecast)

    print("\nDetecting Anomalies...")
    anomalies = detect_pipeline_anomalies(df)
    print(anomalies)

    print("\nGenerating Alerts...")
    alerts = generate_alerts(df, metrics)
    print(alerts)

    print("\nPipeline Completed Successfully.")


if __name__ == "__main__":
    run()


Loading Data...
Adding Features...

Executive Metrics:
{'total_pipeline_value': 131432464, 'avg_deal_size': 26286.4928, 'win_rate': 0.4526, 'avg_sales_cycle': 63.7518}

Training Deal Risk Model...

Running Win Driver Analysis...
                    feature    impact
14      lead_source_Partner -0.075899
1           industry_EdTech -0.066146
3       industry_HealthTech -0.047883
8      region_North America -0.041783
10  product_type_Enterprise -0.041359

Revenue Forecast...
32063625.9836

Detecting Anomalies...
['Unusual high-value deals detected']

Generating Alerts...
['Too many slow-moving deals']

Pipeline Completed Successfully.
